In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import os
from os.path import join
import random
import lightgbm as lgb
import time
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
 
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!nvidia-smi

Mon May 11 13:53:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 432.00       Driver Version: 432.00       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)   WDDM  | 00000000:01:00.0  On |                  N/A |
| 29%   51C    P0    74W / 250W |   1183MiB / 12288MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11673321790902535330
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10074609419
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3430381292429849271
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
def conv_block_3(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def conv_residual(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = conv_block_3(conv1, start_neurons)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def model_v1(input_layer, start_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 14))
output_layer = model_v1(input_layer, 32)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 14)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 5)
img_layer shape: (None, 40, 40, 9)
conv1 shape: (None, 40, 40, 32)
maxpool 1 shape: (None, 20, 20, 32)
conv2 shape: (None, 20, 20, 64)
maxpool 2 shape: (None, 10, 10, 64)
convm shape: (None, 10, 10, 128)
upconv2 shape: (None, 20, 20, 64)
upconv1 shape: (None, 40, 40, 32)
output shape: (None, 40, 40, 1)


In [5]:
# training_set.ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_train():
    # train feather 파일 로드 및 orbit, subset 조합 생성
    TRAIN_FILE = './training_set.ftr'
    train = pd.read_feather(TRAIN_FILE)
    file_cnt = 0
    temp_info = train[['orbit', 'subset']].drop_duplicates()
    orbit = temp_info['orbit'].tolist()
    subset = temp_info['subset'].tolist()
    del temp_info
    size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
    for i in range(size):
        one_img = train.loc[(train['orbit'] == orbit[i]) & (train['subset'] == subset[i])].sort_values('pixel')
        one_img = np.array(one_img.drop(['orbit', 'subset', 'pixel'], axis=1)).reshape([40,40,15])
        target = one_img[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target < 0,0, target)
        feature = one_img[:,:,:-1]
        # if (cutoff_labels > 0).sum() < 50: continue
        yield(feature, cutoff_labels)
        file_cnt += 1

train_dataset = tf.data.Dataset.from_generator(input_generator_ftr_train, (tf.float32, tf.float32),
                                               (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
train_dataset = train_dataset.batch(1024).prefetch(1)

In [6]:
# custom loss fuction (maeOverFscore)
def recall_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    possible_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true, 0, 1))), dtype=tf.float32)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    predicted_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_pred, 0, 1))), dtype=tf.float32)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    idx_one_true = tf.greater_equal(y_true, 0.1)
    y_true = tf.where(idx_one_true == True, 1, 0)
    idx_one_pred = tf.greater_equal(y_pred, 0.1)
    y_pred = tf.where(idx_one_pred == True, 1, 0)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mae(y_true, y_pred):
    over_threshold = tf.greater_equal(y_true, 0.1)
    return K.mean(math_ops.abs(y_pred - y_true), axis=-1)

def maeOverFscore(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return mae(y_true, y_pred) / f1_m(y_true, y_pred)

In [7]:
ROOT_DIR_PATH = os.getcwd()
WEIGHT_DIR = join(ROOT_DIR_PATH,'./checkpoint/')
LOG_DIR = join(ROOT_DIR_PATH, 'log')
# logging = TensorBoard(log_dir=LOG_DIR)

tb_hist = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint(WEIGHT_DIR + 'v01_ep{epoch:02d}-loss{loss:.5f}.ckpt', mode='min',
                             monitor='loss', save_weights_only=True, period=1, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1, verbose=1)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=1)

callbacks = [tb_hist, checkpoint, reduce_lr, early_stopping]
adam = tf.keras.optimizers.Adam(learning_rate=1.0e-07)
model.compile(loss=maeOverFscore, optimizer=adam, metrics=[mae])

In [8]:
# 이전에 훈련한 weight를 이어서 훈련할 경우 실행
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)

C:\Users\User\Documents\not_for_git\orbit_project\./checkpoint/v01_ep04-loss0.86484.ckpt


In [ ]:
start = time.time()
epochs = 14
model_history = model.fit(train_dataset, epochs = epochs, verbose=1,
                          callbacks=callbacks)
print('{}epoch 파일 학습 시간:'.format(epochs), round((time.time()-start)/60, 2))

Epoch 1/14
     24/Unknown - 461s 461s/step - loss: 0.9510 - mae: 0.141 - 896s 448s/step - loss: 0.9845 - mae: 0.147 - 1323s 441s/step - loss: 0.9489 - mae: 0.14 - 1751s 438s/step - loss: 0.9915 - mae: 0.14 - 2190s 438s/step - loss: 0.9517 - mae: 0.14 - 2624s 437s/step - loss: 0.9403 - mae: 0.14 - 3074s 439s/step - loss: 0.9563 - mae: 0.14 - 3506s 438s/step - loss: 0.9390 - mae: 0.14 - 3946s 438s/step - loss: 0.9173 - mae: 0.15 - 4366s 437s/step - loss: 0.9015 - mae: 0.15 - 4787s 435s/step - loss: 0.8905 - mae: 0.16 - 5214s 435s/step - loss: 0.8877 - mae: 0.16 - 5667s 436s/step - loss: 0.8803 - mae: 0.17 - 6091s 435s/step - loss: 0.8704 - mae: 0.17 - 6521s 435s/step - loss: 0.8643 - mae: 0.18 - 6965s 435s/step - loss: 0.8623 - mae: 0.18 - 7407s 436s/step - loss: 0.8587 - mae: 0.18 - 7867s 437s/step - loss: 0.8556 - mae: 0.19 - 8326s 438s/step - loss: 0.8526 - mae: 0.19 - 8764s 438s/step - loss: 0.8494 - mae: 0.19 - 9205s 438s/step - loss: 0.8479 - mae: 0.19 - 9633s 438s/step - loss: 0.

# Cross Validation 평가

In [ ]:
# cv,ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_cv():
    # cv feather 파일 로드 및 orbit, subset 조합 생성
    CV_FILE = './ftr_data/cv.ftr'
    cv = pd.read_feather(CV_FILE)
    file_cnt = 0
    temp_info = cv[['orbit', 'subset']].drop_duplicates()
    orbit = temp_info['orbit'].tolist()
    subset = temp_info['subset'].tolist()
    del temp_info
    size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
    for i in range(size):
        one_img = cv.loc[(cv['orbit'] == orbit[i]) & (cv['subset'] == subset[i])].sort_values('pixel')
        one_img = np.array(one_img.drop(['orbit', 'subset', 'pixel'], axis=1)).reshape([40,40,15])
        target = one_img[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target < 0,0, target)
        feature = one_img[:,:,:-1]
        # if (cutoff_labels > 0).sum() < 50: continue
        yield(feature, cutoff_labels)
        file_cnt += 1

cv_dataset = tf.data.Dataset.from_generator(input_generator_ftr_cv, (tf.float32, tf.float32),
                                            (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
cv_dataset = cv_dataset.batch(64)

In [ ]:
WEIGHT_DIR = './checkpoint/'
LOG_DIR = './log/'
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)
logging = TensorBoard(log_dir=LOG_DIR)
result = model.evaluate(cv_dataset)
dict(zip(model.metrics_names, result))

In [0]:
# fusionNet과 external feature 까지 고려하는 모델 (미완성)
def conv_block_3(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def conv_residual(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = conv_block_3(conv1, start_neurons)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def model_v2(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    print('output shape:', output_layer.shape)

    # process the external feature
    ext_output = Conv2D(1, (1,1), padding="same", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    ext_output = Dropout(0.25)(ext_output)
    ext_output = Conv2D(1, (1,1), padding="same", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)

    output_layer = concatenate(output_layer, ext_output)
    
    return output_layer

input_layer = Input((40, 40, 14))
output_layer = model_v1(input_layer, 32)
model = Model(input_layer, output_layer)